In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Configurar el modelo de embeddings
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


db = Chroma(
    persist_directory="./chroma_db",
    collection_name="info_general",
    embedding_function=embeddings
)

/home/dannylimon/anaconda3/envs/ocr-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
query = "total del documento 35846"
results = db.similarity_search(query=query, k=3)
if not results:
    print("I couldn't find any relevant information in the database.") 

 # Formatear los resultados en un solo texto
context = "\n\n".join([res.page_content for res in results])
# Crear una consulta estructurada para el LLM

In [6]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Crear una consulta estructurada para el LLM
prompt = f"""Based on the following retrieved information, answer the user's query concisely and accurately.
    Retrieved information:
    {context}
    User query: {query}
    """

# Cargar la clave de API desde el archivo .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Configurar el modelo de chat OpenAI con la clave de API cargada
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)

# Crear un mensaje inicial para la conversación
messages = [{"role": "system", "content": "You are a helpful assistant."}]
messages.append({"role": "user", "content": prompt})

# Usar el método 'invoke' para hacer una consulta
response = llm.invoke(messages)

In [7]:
print(response.content)

El total del documento 35846 es de $192.01.
